In [53]:
import pandas as pd
from datetime import datetime

In [54]:
df = pd.read_csv(r"SBS_Processed_Datasets/02_cleaned_output.csv")

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7327 entries, 0 to 7326
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   city             7327 non-null   object 
 1   transmission     7327 non-null   object 
 2   owner_no         7327 non-null   int64  
 3   brand            7327 non-null   object 
 4   model            7327 non-null   object 
 5   model_year       7327 non-null   int64  
 6   variant_name     7327 non-null   object 
 7   price            7327 non-null   float64
 8   registered_year  7327 non-null   int64  
 9   fuel_type        7327 non-null   object 
 10  kms_driven       7327 non-null   float64
 11  mileage_kmpl     7327 non-null   float64
 12  engine_cc        7327 non-null   int64  
dtypes: float64(3), int64(4), object(6)
memory usage: 744.3+ KB


In [56]:
df.head()

,city,transmission,owner_no,brand,model,model_year,variant_name,price,registered_year,fuel_type,kms_driven,mileage_kmpl,engine_cc
0,bangalore,manual,3,maruti,maruti celerio,2015,vxi,400000.0,2015,petrol,120000.0,23.10,998
1,bangalore,manual,2,ford,ford ecosport,2018,1.5 petrol titanium bsiv,811000.0,2018,petrol,32706.0,17.00,1497
2,bangalore,manual,1,tata,tata tiago,2018,1.2 revotron xz,585000.0,2018,petrol,11949.0,23.84,1199
3,bangalore,manual,1,hyundai,hyundai xcent,2014,1.2 kappa s option,462000.0,2014,petrol,17794.0,19.10,1197
4,bangalore,manual,1,maruti,maruti sx4 s cross,2015,ddis 200 zeta,790000.0,2015,diesel,60000.0,23.65,1248


FEATURE ENGINEERING 

In [57]:
#STEP_1 ADD AGE RELATED FEATURES

# Get the current year
current_year = datetime.now().year

# Calculate car age
df['car_age'] = current_year - df['registered_year']
df['model_age'] = current_year - df['model_year']
df['registration_lag']  = df['registered_year'] - df['model_year'] 
df['normalized_model_age']  = df['car_age'] / (df['model_age']  + 1)


In [58]:
#STEP_2 Add price_per_km

# Calculate price per kilometer driven
df['price_per_km'] = df['price'] / df['kms_driven']
df['price_per_km'] = df['price_per_km'].replace([float('inf'), -float('inf')], 0)  # Replace infinities with 0
df['price_per_km'] = df['price_per_km'].fillna(0)

In [59]:
#STEP_3 Create mileage_normalized

# Calculate normalized mileage
df['mileage_normalized'] = df['mileage_kmpl'] / df['mileage_kmpl'].max()

In [60]:
#STEP_4 High Mileage Flag

# high_mileage (binary feature)
high_mileage_threshold = 150000  # Define the threshold for high mileage
df['high_mileage'] = (df['kms_driven'] > high_mileage_threshold).astype(int)

In [61]:
#STEP_5 Multiple Owners

# Flag for cars with multiple previous owners
df['multiple_owners'] = (df['owner_no'] > 1).astype(int)


In [62]:
#STEP_6 Brand Popularity

# Compute average price per brand
brand_avg_price = df.groupby('brand')['price'].mean().to_dict()

# Map the average price to each row
df['brand_popularity'] = df['brand'].map(brand_avg_price)

# Example: Adding brand popularity column
print(df[['brand', 'brand_popularity']].head())


     brand  brand_popularity
0   maruti     503886.023982
1     ford     538652.329749
2     tata     617492.501340
3  hyundai     575329.531686
4   maruti     503886.023982


In [63]:
#STEP_7 kms_per_year
df['kms_per_year'] = df['kms_driven'] / (df['car_age'] + 1)


In [64]:
df.head(2)

,city,transmission,owner_no,brand,model,model_year,variant_name,price,registered_year,fuel_type,...,car_age,model_age,registration_lag,normalized_model_age,price_per_km,mileage_normalized,high_mileage,multiple_owners,brand_popularity,kms_per_year
0,bangalore,manual,3,maruti,maruti celerio,2015,vxi,400000.0,2015,petrol,...,9,9,0,0.900000,3.333333,0.648876,0,1,503886.023982,12000.000000
1,bangalore,manual,2,ford,ford ecosport,2018,1.5 petrol titanium bsiv,811000.0,2018,petrol,...,6,6,0,0.857143,24.796673,0.477528,0,1,538652.329749,4672.285714


In [65]:
df.describe()

,owner_no,model_year,price,registered_year,kms_driven,mileage_kmpl,engine_cc,car_age,model_age,registration_lag,normalized_model_age,price_per_km,mileage_normalized,high_mileage,multiple_owners,brand_popularity,kms_per_year
count,7327.000000,7327.000000,7.327000e+03,7327.000000,7.327000e+03,7327.000000,7327.000000,7327.000000,7327.000000,7327.000000,7327.000000,7327.000000,7327.000000,7327.000000,7327.000000,7.327000e+03,7327.000000
mean,1.380920,2016.213321,6.260850e+05,2016.279924,6.040579e+04,19.595483,1341.502525,7.720076,7.786679,0.066603,0.849895,24.173851,0.550435,0.007643,0.305992,6.260850e+05,7227.724674
std,0.647962,3.887329,3.561220e+05,3.898677,4.438606e+04,3.484031,381.741087,3.898677,3.887329,0.323250,0.085593,165.732636,0.097866,0.087095,0.460858,2.085202e+05,5903.800914
min,1.000000,1985.000000,2.800000e+04,1985.000000,1.010000e+02,7.080000,624.000000,1.000000,1.000000,0.000000,0.105263,0.248711,0.198876,0.000000,0.000000,7.333333e+04,50.500000
25%,1.000000,2014.000000,3.720000e+05,2014.000000,3.343100e+04,17.380000,1197.000000,5.000000,5.000000,0.000000,0.833333,5.416667,0.488202,0.000000,0.000000,5.038860e+05,4566.250000
50%,1.000000,2017.000000,5.510000e+05,2017.000000,5.667200e+04,19.160000,1199.000000,7.000000,7.000000,0.000000,0.875000,10.045467,0.538202,0.000000,0.000000,5.753295e+05,6581.333333
75%,2.000000,2019.000000,8.000000e+05,2019.000000,8.000000e+04,21.740000,1497.000000,10.000000,10.000000,0.000000,0.909091,20.643265,0.610674,0.000000,1.000000,6.210198e+05,9095.306818
max,5.000000,2023.000000,1.775000e+06,2023.000000,2.000022e+06,35.600000,5000.000000,39.000000,39.000000,16.000000,0.975000,13069.306931,1.000000,1.000000,1.000000,1.725000e+06,250002.750000


In [66]:
df.head(2)

,city,transmission,owner_no,brand,model,model_year,variant_name,price,registered_year,fuel_type,...,car_age,model_age,registration_lag,normalized_model_age,price_per_km,mileage_normalized,high_mileage,multiple_owners,brand_popularity,kms_per_year
0,bangalore,manual,3,maruti,maruti celerio,2015,vxi,400000.0,2015,petrol,...,9,9,0,0.900000,3.333333,0.648876,0,1,503886.023982,12000.000000
1,bangalore,manual,2,ford,ford ecosport,2018,1.5 petrol titanium bsiv,811000.0,2018,petrol,...,6,6,0,0.857143,24.796673,0.477528,0,1,538652.329749,4672.285714


In [67]:
df.columns

Index(['city', 'transmission', 'owner_no', 'brand', 'model', 'model_year',
       'variant_name', 'price', 'registered_year', 'fuel_type', 'kms_driven',
       'mileage_kmpl', 'engine_cc', 'car_age', 'model_age', 'registration_lag',
       'normalized_model_age', 'price_per_km', 'mileage_normalized',
       'high_mileage', 'multiple_owners', 'brand_popularity', 'kms_per_year'],
      dtype='object')

In [68]:
# Export it
df.to_csv(r"SBS_Processed_Datasets\03_feature_engineered_output.csv",index=False)